In [1]:
if 'spark0' not in globals():
    spark0 = spark
    spark = SparkSession.builder \
        .config("spark.sql.hive.convertMetastoreParquet", "false") \
        .config("hive.metastore.uris", "thrift://metastore.data.hotstar-labs.com:9083") \
        .enableHiveSupport() \
        .getOrCreate()

In [4]:
import os
from datetime import datetime
import pyspark.sql.functions as F
import pandas as pd

out_path = 's3://adtech-ml-perf-ads-us-east-1-prod-v1/live_inventory_forecasting/data/subs_AU/'
order_path = 's3://hotstar-ads-ml-us-east-1-prod/content-insight/content_return/subs_retention/data/order_base/'
AU_table = 'data_warehouse.viewed_page_daily_aggregates_ist'

def s3_exist(path):
    return 0 == os.system(f'aws s3 ls {path}')

In [5]:
order = spark.read.parquet(order_path)

In [ ]:
month_cache_enabled = False
cache_month = ''

for i in pd.date_range('2022-01-01', '2023-01-01'):
    cd = str(i.date())
    real_path = f'{out_path}cd={cd}/'
    print(datetime.now(), real_path)
    if s3_exist(real_path + '_SUCCESS'):
        continue
    if month_cache_enabled:
        month = cd[:7]
        if cache_month != month:
            print(11)
            if 'order2' in globals():
                print(22)
                order2.unpersist()
            print(33)
            order2 = order.where(f'substr(order_start_date, 1, 7) <= "{cache_month}" and substr(actual_order_end_date, 1, 7) >= "{cache_month}"')
            print(order2.count())
            cache_month = month
    else:
        order2 = order
    vp = spark.sql(f'select dw_p_id from {AU_table} where cd = "{cd}"')
    order3 = order2.where(f'order_start_date <= "{cd}" and actual_order_end_date >= "{cd}" and is_cancel = 0')
    vp.join(order3, 'dw_p_id').groupby('partner', 'billing_interval_unit', 'billing_frequency').agg(F.countDistinct('dw_p_id').alias('n_pid')) \
        .repartition(1).write.mode('overwrite').parquet(real_path)
    print(datetime.now())